In [1]:
import warnings
import time

import pandas as pd

from experiment.utils import dbutils, logger, transformation
from experiment.utils.tables.upload_tasks_table import UploadTasksTable

* Pipeline with ChatGPT
* Remove spaces between sentence

In [3]:
db = dbutils.DatabaseUtils()
lg = logger.Logger(
    logging_level="DEBUG",
    output_path=transformation.get_project_root() / "tmp" / "report_prompting.log",
)

In [12]:
reports_raw, Base = UploadTasksTable()
Base.metadata.create_all(db.engine)

In [4]:
TRANSLATE_N_MORE_REPORTS = 1

In [5]:
# get reports directly from database
query = """
            SELECT * FROM annotation.upload_tasks
        """

# get values from the database
df_reports = db.read_sql_query(query)
df_reports.head()

,report_id,patient_no,protocol_no,report_original,report_length,report_prompted,patient_report_count
0,1,2004001412,23012646,RAPOR TARİHİ: 26/09/2022 FİLM NO: 12796199\n...,70,None,1
1,2,2004001562,22394628,RAPOR TARİHİ: 15.03.2022 FİLM NO: 12523533\...,66,None,1
2,3,2004001709,22296380,RAPOR TARİHİ:15/02/2022 FİLM NO:12493333\n\n...,74,None,1
3,4,2004002163,23963976,RAPOR TARİHİ: 25/05/2023 FİLM NO:\n\nKontras...,80,None,1
4,5,2004002609,21367900,RAPOR TARİHİ: 09.08.2021 FİLM NO: 11338850...,63,None,1


In [6]:
# get annotated reports 
query = """
            SELECT 
                DISTINCT data ->> 'patient_no' as patient_no
            FROM task
            WHERE is_labeled = TRUE
        """

# get values from the database
annotated_patient_nos = db.read_sql_query(query)["patient_no"].to_list()

In [7]:
# get tasks that have been prompted
query = """
            SELECT 
                report_id
            FROM annotation.upload_tasks
            WHERE report_prompted IS NOT NULL 
        """

# get values from the database
upload_tasks_prompted = db.read_sql_query(query)["report_id"].to_list()

In [9]:
# use only non-prompted reports & non-annotated patients
df_upload_tasks = (
    df_reports.loc[~df_reports["patient_no"].isin(annotated_patient_nos)]
    .loc[~df_reports["report_id"].isin(upload_tasks_prompted)]
    .head(TRANSLATE_N_MORE_REPORTS)
)

In [10]:
df_upload_tasks.columns

Index(['report_id', 'patient_no', 'protocol_no', 'report_original',
       'report_length', 'report_prompted', 'patient_report_count'],
      dtype='object')

In [11]:
cols_to_upsert = df_upload_tasks.columns.to_list()
cols_to_upsert.remove("report_id")
data_to_insert = []
for _, row in df_upload_tasks.iterrows():

    data_to_insert.append(
        {
            "report_id": row["report_id"],
            "patient_no": row["patient_no"],
            "protocol_no": row["protocol_no"],
            "report_original": row["report_original"],
            "report_prompted":  "tests", # transformation.prompt_report(row["report_prompted"]),
            "report_length": row["report_length"],
            "patient_report_count": row["patient_report_count"]
        }
    )

    db.upsert_values(reports_raw, data_to_insert, cols_to_upsert, ["report_id"])

    # openai restriction: 3 RPM - 200 RPD
    time.sleep(20)